# ifctruss

> Core function of IfcTruss

In [ ]:
# | default_exp ifctruss

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc
import copy

In [ ]:
# | export
from collections import namedtuple
from typing import NamedTuple
import ifcopenshell
import pandas as pd

In [ ]:
# | export
import ifctruss._structural_analysis_model
import ifctruss._view
import ifctruss._save_result
import ifctruss._example
import ifctruss.solver

## Create spreadsheet template

In [ ]:
# | hide
# | export
_metadata = pd.DataFrame(
    ["https://github.com/kulasegaram/IfcTruss",
     ""], columns=["Metadata"], index=["Repo", ""]
)

In [ ]:
# | export
def save_ods_template(
    ods_path: str = "ifctruss-template.ods",  # Path to ods file
    load_groups: bool = False,  # Create worksheet for information regarding IfcStructuralLoadGroup's
):
    with pd.ExcelWriter(ods_path, engine="odf") as writer:
        _metadata.to_excel(writer, sheet_name="IfcTruss", index=True)
        ifctruss._example.nodes.to_excel(
            writer, sheet_name="Nodes", index=False
        )
        ifctruss._example.bars.to_excel(
            writer, sheet_name="Bars", index=False
        )
        ifctruss._example.point_loads.to_excel(
            writer, sheet_name="Point_Loads", index=False
        )
        if load_groups:
            ifctruss._example.load_groups.to_excel(
                writer, sheet_name="Load_Groups", index=False
            )

::: {.callout-note}
If `load_groups` is `False`, no worksheet `Load_Groups` will be created.
:::

In [ ]:
# | hide
save_ods_template()

In [ ]:
# | export
def save_xlsx_template(
    xlsx_path: str = "ifctruss-template.xlsx",  # Path to xlsx file
    load_groups: bool = False,  # Create worksheet for information regarding IfcStructuralLoadGroup's
):
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
        _metadata.to_excel(writer, sheet_name="IfcTruss", index=True)
        ifctruss._example.nodes.to_excel(
            writer, sheet_name="Nodes", index=False
        )
        ifctruss._example.bars.to_excel(
            writer, sheet_name="Bars", index=False
        )
        ifctruss._example.point_loads.to_excel(
            writer, sheet_name="Point_Loads", index=False
        )
        if load_groups:
            ifctruss._example.load_groups.to_excel(
                writer, sheet_name="Load_Groups", index=False
            )

::: {.callout-note}
If `load_groups` is `False`, no worksheet `Load_Groups` will be created.
:::

In [ ]:
# | hide
save_xlsx_template()

## Build a IFC

In [ ]:
# | export
def build(
    nodes: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralPointConnection's
    bars: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralCurveMember's
    point_loads: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralLoadSingleForce's
    load_groups: pd.DataFrame
    | None = None,  # Pandas DataFrame with information regarding IfcStructuralLoadGroup's
) -> ifcopenshell.file:  # IFC model
    if load_groups is None:
        # Create one load group and put all point loads into it
        load_groups = point_loads[["Point_Load"]].copy()
        load_groups["Load_Group"] = 1

    truss = (
        ifctruss._structural_analysis_model.StructuralAnalysisModelCreation()
    )
    truss.create_basic_structure()
    truss.create_from_DataFrame_a_truss_model(
        Bars=bars,
        Nodes=nodes,
        Point_Loads=point_loads,
        Load_Groups=load_groups,
    )
    return truss.ifc_model

::: {.callout-note}
If `load_groups` is `None`, only one IfcStructuralLoadGroup is created and all IfcStructuralLoadSingleForce are related to it.
:::

In [ ]:
# | hide
import ifcopenshell.validate
from rich import print

In [ ]:
# | hide
ifc_model = build(**ifctruss._example.dfs)

In [ ]:
# | hide
json_logger = ifcopenshell.validate.json_logger()
ifcopenshell.validate.validate(ifc_model, json_logger)
json_list = json_logger.statements
if json_list:
    for i in json_list:
        print(i)

In [ ]:
# | hide
# An empty list will evaluate to False.
assert not json_list

In [ ]:
# | export
def build_from_ods(
    ods_path: str,  # Path to ods file
) -> ifcopenshell.file:  # IFC model
    odf = pd.ExcelFile(ods_path, engine="odf")

    Nodes_dtype = {
        "Coordinate_X": float,
        "Coordinate_Y": float,
        "Coordinate_Z": float,
        "Translational_X": bool,
        "Translational_Y": bool,
        "Translational_Z": bool,
    }
    # try:
    Nodes = odf.parse("Nodes", dtype=Nodes_dtype)

    Bars_dtype = {
        "Cross-sectional_area": float,
        "Modulus_of_elasticity": float,
    }
    # try:
    Bars = odf.parse("Bars", dtype=Bars_dtype)

    Point_Loads_dtype = {
        "Force_X": float,
        "Force_Y": float,
        "Force_Z": float,
    }
    # try:
    Point_Loads = odf.parse("Point_Loads", dtype=Point_Loads_dtype)

    if "Load_Groups" in odf.sheet_names:
        # try:
        Load_Groups = odf.parse("Load_Groups")
    else:
        Load_Groups = None

    ifc_model = build(
        bars=Bars,
        nodes=Nodes,
        point_loads=Point_Loads,
        load_groups=Load_Groups,
    )
    return ifc_model

::: {.callout-note}
If there is no worksheet `Load_Groups`, only one IfcStructuralLoadGroup is created and all IfcStructuralLoadSingleForce are related to it.
:::

In [ ]:
# | hide
ifc_model = build_from_ods("ifctruss-template.ods")

In [ ]:
# | export
def build_from_xlsx(
    xlsx_path: str,  # Path to xlsx file
) -> ifcopenshell.file:  # IFC model
    xlsx = pd.ExcelFile(xlsx_path, engine="openpyxl")

    Nodes_dtype = {
        "Coordinate_X": float,
        "Coordinate_Y": float,
        "Coordinate_Z": float,
        "Translational_X": bool,
        "Translational_Y": bool,
        "Translational_Z": bool,
    }
    # try:
    Nodes = xlsx.parse("Nodes", dtype=Nodes_dtype)

    Bars_dtype = {
        "Cross-sectional_area": float,
        "Modulus_of_elasticity": float,
    }
    # try:
    Bars = xlsx.parse("Bars", dtype=Bars_dtype)

    Point_Loads_dtype = {
        "Force_X": float,
        "Force_Y": float,
        "Force_Z": float,
    }
    # try:
    Point_Loads = xlsx.parse("Point_Loads", dtype=Point_Loads_dtype)

    if "Load_Groups" in xlsx.sheet_names:
        # try:
        Load_Groups = xlsx.parse("Load_Groups")
    else:
        Load_Groups = None

    ifc_model = build(
        bars=Bars,
        nodes=Nodes,
        point_loads=Point_Loads,
        load_groups=Load_Groups,
    )
    return ifc_model

::: {.callout-note}
If there is no worksheet `Load_Groups`, only one IfcStructuralLoadGroup is created and all IfcStructuralLoadSingleForce are related to it.
:::

In [ ]:
# | hide
ifc_model = build_from_xlsx("ifctruss-template.xlsx")

## View as DataFrame

In [ ]:
# | export
def _determination_of_structural_analysis_model_load_group(
    model,
    structural_analysis_model,
    load_group,
):
    class IfcStructuralAnalysisModelException(Exception):
        pass

    if structural_analysis_model is None:
        structural_analysis_models = model.by_type(
            "IfcStructuralAnalysisModel"
        )
        structural_analysis_models_count = len(structural_analysis_models)
        if structural_analysis_models_count == 0:
            raise IfcStructuralAnalysisModelException(
                "No IfcStructuralAnalysisModel found"
            )
        elif structural_analysis_models_count == 1:
            structural_analysis_model = structural_analysis_models[
                0
            ].GlobalId
        elif structural_analysis_models_count > 1:
            raise IfcStructuralAnalysisModelException(
                f"Found {structural_analysis_models_count} IfcStructuralAnalysisModel - please provide the GlobalId"
            )

    class IfcStructuralLoadGroupException(Exception):
        pass

    if load_group is None:
        load_groups = model.by_type("IfcStructuralLoadGroup")
        load_groups_count = len(load_groups)
        if load_groups_count == 0:
            raise IfcStructuralAnalysisModelException(
                "No IfcStructuralLoadGroup found"
            )
        elif load_groups_count == 1:
            load_group = load_groups[0].GlobalId
        elif load_groups_count > 1:
            raise IfcStructuralLoadGroupException(
                f"Found {load_groups_count} IfcStructuralLoadGroup - please provide the GlobalId"
            )

    return structural_analysis_model, load_group

In [ ]:
# | export
def view(
    model: ifcopenshell.file,  # IFC model
    structural_analysis_model: str
    | None = None,  # str: IfcStructuralAnalysisModel GlobalId
    load_group: str | None = None,  # str: IfcStructuralLoadGroup GlobalId
    result_group: str
    | bool = False,  # str: IfcStructuralResultGroup GlobalId
) -> NamedTuple:  # NamedTuple with Pandas DataFrame's (and str's)
    (
        structural_analysis_model,
        load_group,
    ) = _determination_of_structural_analysis_model_load_group(
        model, structural_analysis_model, load_group
    )

    view_object = ifctruss._view.View(
        model=model,
        structural_analysis_model=structural_analysis_model,
        load_group=load_group,
    )

    view_object.get_nodes_and_bars()
    nodes_df = view_object.nodes_df
    bars_df = view_object.bars_df

    view_object.get_point_loads()
    point_loads_df = view_object.point_loads_df

    if not result_group:
        dfs = namedtuple("df", "nodes bars point_loads")

        return dfs(nodes_df, bars_df, point_loads_df)

    elif result_group and not isinstance(result_group, str):

        class IfcStructuralResultGroupException(Exception):
            pass

        result_groups = model.by_type("IfcStructuralResultGroup")
        result_groups_count = len(result_groups)
        if result_groups_count == 0:
            raise IfcStructuralResultGroupException(
                "No IfcStructuralResultGroup found"
            )
        elif result_groups_count == 1:
            result_group = result_groups[0].GlobalId
        elif result_groups_count > 1:
            raise IfcStructuralResultGroupException(
                f"Found {result_groups_count} IfcStructuralResultGroup - please provide the GlobalId"
            )

    view_object.get_result_group(result_group)
    theory_type = view_object.theory_type
    is_linear = view_object.is_linear

    view_object.get_displacements()
    displacments_df = view_object.displacments_df

    view_object.get_forces()
    forces_df = view_object.forces_df

    view_object.get_normal_forces()
    normal_forces_df = view_object.normal_forces_df

    dfs = namedtuple(
        "df",
        "nodes bars point_loads displacments forces normal_forces theory_type is_linear",
    )
    return dfs(
        nodes_df,
        bars_df,
        point_loads_df,
        displacments_df,
        forces_df,
        normal_forces_df,
        theory_type,
        is_linear,
    )

::: {.callout-note}
If `structural_analysis_model=None` or `load_group=None` or `result_group=True`: IfcStructuralAnalysisModel or IfcStructuralLoadGroup or IfcStructuralResultGroup will be searched and if there is only one possibility, also used, if not, the user will be requested to provide the respective GlobalId. 
If `result_group` is `False`, there will be no attempt to fetch the information regarding the IfcStructuralResultGroup.
:::

In [ ]:
# | hide
nodes, bars, point_loads = view(ifc_model)

In [ ]:
# | hide
dfs = view(ifc_model)

## Calculate and save result in IFC

In [ ]:
# | export
def save_result(
    model: ifcopenshell.file,  # IFC model
    displacments: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralPointReaction IfcStructuralLoadSingleDisplacement
    forces: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralPointReaction IfcStructuralLoadSingleForce
    normal_forces: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralCurveReaction
    theory_type: str,  # IfcStructuralResultGroup TheoryType
    is_linear: str,  # IfcStructuralResultGroup IsLinear
    structural_analysis_model: str
    | None = None,  # str: IfcStructuralAnalysisModel GlobalId
    load_group: str | None = None,  # str: IfcStructuralLoadGroup GlobalId
):
    (
        structural_analysis_model,
        load_group,
    ) = _determination_of_structural_analysis_model_load_group(
        model, structural_analysis_model, load_group
    )

    save_result_object = ifctruss._save_result.SaveResult(
        model=model,
        structural_analysis_model=structural_analysis_model,
        load_group=load_group,
        theory_type=theory_type,
        is_linear=is_linear,
        displacments=displacments,
        forces=forces,
        normal_forces=normal_forces,
    )
    save_result_object.create_structural_result_group()
    save_result_object.create_displacments()
    save_result_object.create_forces()
    save_result_object.create_normal_forces()
    save_result_object.assign_to_the_result_group()

::: {.callout-note}
If `structural_analysis_model=None` or `load_group=None`: IfcStructuralAnalysisModel or IfcStructuralLoadGroup will be searched and if there is only one possibility, also used, if not, the user will be requested to provide the respective GlobalId.
:::

In [ ]:
# | export
def solve(
    model: ifcopenshell.file,  # IFC model
    structural_analysis_model=None,  # str: IfcStructuralAnalysisModel GlobalId
    load_group=None,  # str: IfcStructuralLoadGroup GlobalId
    solver="direct_stiffness_method",  # str: ["direct_stiffness_method", "calfem"]
):
    solver = solver.lower()
    (
        structural_analysis_model,
        load_group,
    ) = _determination_of_structural_analysis_model_load_group(
        model, structural_analysis_model, load_group
    )

    dfs = view(
        model,
        structural_analysis_model=structural_analysis_model,
        load_group=load_group,
    )._asdict()

    map_name_to_solver = {
        "direct_stiffness_method": ifctruss.solver.direct_stiffness_method,
        "calfem": ifctruss.solver.calfem,
    }
    if solver not in map_name_to_solver:
        pass  # raise something

    results = map_name_to_solver[solver](**dfs)._asdict()

    # save result in ifc
    save_result(
        model,
        **results,
        structural_analysis_model=structural_analysis_model,
        load_group=load_group,
    )

::: {.callout-note}
If `structural_analysis_model=None` or `load_group=None`: IfcStructuralAnalysisModel or IfcStructuralLoadGroup will be searched and if there is only one possibility, also used, if not, the user will be requested to provide the respective GlobalId.
:::

In [ ]:
# | hide
ifc_model.write("truss.ifc")
solve(ifc_model)

In [ ]:
# | hide
model_calfem = ifcopenshell.open("truss.ifc")

In [ ]:
# | hide
solve(model_calfem, solver="calfem")

In [ ]:
# | hide
view_dfs = view(ifc_model, result_group=True)

In [ ]:
# | hide
view_dfs = view(model_calfem, result_group=True)

IfcStructuralResultGroupException: Found 2 IfcStructuralResultGroup - please provide the GlobalId

In [ ]:
# | hide
nbdev.nbdev_export()